In [47]:
import gensim
word2vecmodel = gensim.models.KeyedVectors.load_word2vec_format(path.expanduser('~/work/jupyter_notebooks/notebooks/GoogleNews-vectors-negative300.bin'), binary=True)

In [49]:
word2vecmodel.save(path.expanduser('~/work/data/word2vec.bin'))

In [52]:
word2vecmodel = gensim.models.KeyedVectors.load(path.expanduser('~/work/data/word2vec.bin'), mmap='r')

In [43]:
import requests
url = 'https://www.ted.com/talks/craig_costello_cryptographers_quantum_computers_and_the_war_for_information'
title = url.split('/')[-1]
metaURL = 'https://api.ted.com/v1/talks/{}.json?api-key=uzdyad5pnc2mv2dd8r8vd65c'
r = requests.get(url = metaURL.format(title))
data = r.json()
id = data['talk']['id']
# [tag['tag'] for tag in data['talk']['tags']]

50956

In [10]:
import numpy as np
import pickle
import pandas as pd
from os import path
import re

with open(path.expanduser('~/work/data/data.pkl'), "rb") as input_file:
    data = pickle.load(input_file)
    
tags = [re.sub('\'','',taglist)[1:-1].split(', ') for taglist in data['tags'].values]
file = open(path.expanduser('~/work/data/tags.txt'), "w")
for talktags in tags:
    file.write(' '.join(talktags) + '\n')
file.close()

In [36]:
import requests
import numpy as np
import nltk
import re
import gensim
nltk.download('stopwords')
nltk.download('wordnet')

def retrieve_prepare_subtitles(url):
    
    metaDataURL = 'https://api.ted.com/v1/talks/{}.json?api-key=uzdyad5pnc2mv2dd8r8vd65c'
    subURL = 'https://api.ted.com/v1/talks/{}/subtitles.json?api-key=uzdyad5pnc2mv2dd8r8vd65c'
    r = requests.get(url = subURL.format(id))
    data = r.json() 
    transcript = []
    for dict in np.arange(len(data)-1):
        text = data[str(dict)]['caption']['content']
        text = re.sub(r'\((.*?)\)', ' ', text)
        text = re.sub(r'\d+', '', text)
        text = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text.lower())
        text = [w for w in text if w not in nltk.corpus.stopwords.words('english')]
        text = [nltk.corpus.wordnet.morphy(w) if nltk.corpus.wordnet.morphy(w) else w for w in text]
        transcript.extend(text)
    return ' '.join(transcript)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:
import os

file = open(path.expanduser('~/work/test/lftm/doc.txt'), "w")
file.write(retrieve_prepare_subtitles(176)) 
file.close()

model = 'LFLDAinf'
paras = 'TEDLFLDA.paras'
corpus = 'doc.txt'
initer = '500'
niter = '50'
topn = '20'
name = 'TEDLFLDAinf'
sstep = '0'

os.chdir(os.path.expanduser('~/work/test/lftm/'))
os.system('java -jar jar/LFTM.jar -model {} -paras {} -corpus {} -initers {} -niters {} -twords {} -name {} -sstep {}'.format(
    model,
    paras,
    corpus,
    initer,
    niter,
    topn,
    name,
    sstep))

0

In [56]:
file = open(path.expanduser('~/work/test/lftm/TEDLFLDAinf.theta'), "r")
topics = file.readline()
file.close()

In [ ]:
def train(datapath):
    text = []
    with open(datapath,"r+") as f:
        while True:
            vec = f.readline()
            text.append(text)

In [2]:
import pickle
from os import path
import pandas as pd

with open(path.expanduser('~/work/data/data.pkl'), "rb") as input_file:
    data = pickle.load(input_file)

In [83]:
import json
import requests
import pandas as pd
import numpy as np

metaDataURL = 'https://api.ted.com/v1/talks/{}.json?api-key=uzdyad5pnc2mv2dd8r8vd65c'
subURL = 'https://api.ted.com/v1/talks/{}/subtitles.json?api-key=uzdyad5pnc2mv2dd8r8vd65c'
_id=1

talk_meta = pd.DataFrame()
talk_sub = pd.DataFrame()
talk = {}
sub = {}
while True:
    response_meta = requests.get(metaDataURL.format(_id))
    response_meta = response_meta.json()
    if 'error' in response_meta:
        print("Id",_id, "not found.")
        _id += 1
        continue
    response_sub = requests.get(subURL.format(_id))
    response_sub = response_sub.json()
    if 'error' in response_sub:
        print("Sub for",_id, "not found.")
        _id += 1
        continue
    talk['talk_id'] = _id
    talk['name'] = response_meta['talk']['name']
    talk['tags'] = ','.join([tag['tag'] for tag in response_meta['talk']['tags']])
    talk_meta = talk_meta.append(talk, ignore_index=True)
    
    chapter_id = 0
    chapter = []
    
    no_caption = len(response_sub)
    chapter.append(response_sub['0']['caption']['content'])
    
    for dict in np.arange(2,no_caption - 2):
        if response_sub[str(dict)]['caption']['startOfParagraph']:
            sub['talk_id'] = _id
            sub['id'] = chapter_id
            sub['transcript'] = ' '.join(chapter)
            talk_sub = talk_sub.append(sub, ignore_index=True)
            chapter_id += 1
            chapter = []
            chapter.append(response_sub[str(dict)]['caption']['content'])
        else:
            chapter.append(response_sub[str(dict)]['caption']['content'])
            
    if response_sub[str(no_caption - 2)]['caption']['startOfParagraph']:
        sub['talk_id'] = _id
        sub['id'] = chapter_id
        sub['transcript'] = ' '.join(chapter)
        talk_sub = talk_sub.append(sub, ignore_index=True)
        chapter_id += 1
        chapter = []
        chapter.append(response_sub[str(no_caption - 2)]['caption']['content'])
        sub['talk_id'] = _id
        sub['id'] = chapter_id
        sub['transcript'] = ' '.join(chapter)
        talk_sub = talk_sub.append(sub, ignore_index=True)
    else:
        sub['talk_id'] = _id
        sub['id'] = chapter_id
        chapter.append(response_sub[str(no_caption - 2)]['caption']['content'])
        sub['transcript'] = ' '.join(chapter)
        talk_sub = talk_sub.append(sub, ignore_index=True)
                
#         text = re.sub(r'\((.*?)\)', ' ', text)
#         text = re.sub(r'\d+', '', text)
#         text = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text.lower())
#         text = [w for w in text if w not in nltk.corpus.stopwords.words('english')]
#         text = [nltk.corpus.wordnet.morphy(w) if nltk.corpus.wordnet.morphy(w) else w for w in text]
#         transcript.extend(text)
    print("Loaded talk",_id)
    _id += 1

Loaded talk 1
Loaded talk 2
Loaded talk 3
Loaded talk 4
Loaded talk 5
Loaded talk 6
Loaded talk 7
Loaded talk 8
Loaded talk 9
Loaded talk 10
Loaded talk 11
Loaded talk 12
Loaded talk 13
Loaded talk 14
Id 15 not found.
Loaded talk 16
Id 17 not found.
Loaded talk 18
Loaded talk 19
Loaded talk 20
Loaded talk 21
Loaded talk 22
Loaded talk 23
Sub for 24 not found.
Loaded talk 25
Loaded talk 26
Loaded talk 27
Loaded talk 28
Loaded talk 29
Loaded talk 30
Loaded talk 31
Loaded talk 32
Loaded talk 33
Loaded talk 34
Loaded talk 35
Loaded talk 36
Loaded talk 37
Loaded talk 38
Loaded talk 39
Loaded talk 40
Loaded talk 41
Loaded talk 42
Loaded talk 43
Loaded talk 44
Loaded talk 45
Loaded talk 46
Loaded talk 47
Loaded talk 48
Loaded talk 49
Loaded talk 50
Loaded talk 51
Id 52 not found.
Loaded talk 53
Loaded talk 54
Loaded talk 55
Loaded talk 56
Loaded talk 57
Loaded talk 58
Loaded talk 59
Loaded talk 60
Loaded talk 61
Loaded talk 62
Loaded talk 63
Loaded talk 64
Loaded talk 65
Loaded talk 66
Loaded

Loaded talk 506
Loaded talk 507
Id 508 not found.
Loaded talk 509
Loaded talk 510
Loaded talk 511
Loaded talk 512
Id 513 not found.
Id 514 not found.
Loaded talk 515
Loaded talk 516
Loaded talk 517
Loaded talk 518
Loaded talk 519
Loaded talk 520
Loaded talk 521
Id 522 not found.
Loaded talk 523
Loaded talk 524
Loaded talk 525
Loaded talk 526
Loaded talk 527
Id 528 not found.
Loaded talk 529
Id 530 not found.
Loaded talk 531
Loaded talk 532
Loaded talk 533
Loaded talk 534
Loaded talk 535
Id 536 not found.
Loaded talk 537
Loaded talk 538
Id 539 not found.
Loaded talk 540
Sub for 541 not found.
Id 542 not found.
Id 543 not found.
Loaded talk 544
Loaded talk 545
Id 546 not found.
Loaded talk 547
Loaded talk 548
Loaded talk 549
Id 550 not found.
Loaded talk 551
Loaded talk 552
Loaded talk 553
Loaded talk 554
Loaded talk 555
Loaded talk 556
Loaded talk 557
Loaded talk 558
Id 559 not found.
Loaded talk 560
Loaded talk 561
Loaded talk 562
Loaded talk 563
Id 564 not found.
Loaded talk 565
Loade

Loaded talk 993
Loaded talk 994
Loaded talk 995
Loaded talk 996
Loaded talk 997
Loaded talk 998
Id 999 not found.
Loaded talk 1000
Loaded talk 1001
Loaded talk 1002
Loaded talk 1003
Loaded talk 1004
Loaded talk 1005
Loaded talk 1006
Loaded talk 1007
Loaded talk 1008
Loaded talk 1009
Loaded talk 1010
Loaded talk 1011
Loaded talk 1012
Loaded talk 1013
Loaded talk 1014
Loaded talk 1015
Loaded talk 1016
Loaded talk 1017
Loaded talk 1018
Loaded talk 1019
Loaded talk 1020
Id 1021 not found.
Id 1022 not found.
Id 1023 not found.
Id 1024 not found.
Id 1025 not found.
Id 1026 not found.
Id 1027 not found.
Id 1028 not found.
Id 1029 not found.
Loaded talk 1030
Loaded talk 1031
Loaded talk 1032
Loaded talk 1033
Loaded talk 1034
Id 1035 not found.
Loaded talk 1036
Loaded talk 1037
Sub for 1038 not found.
Loaded talk 1039
Loaded talk 1040
Loaded talk 1041
Loaded talk 1042
Loaded talk 1043
Loaded talk 1044
Loaded talk 1045
Loaded talk 1046
Loaded talk 1047
Loaded talk 1048
Loaded talk 1049
Loaded ta

Loaded talk 1469
Loaded talk 1470
Loaded talk 1471
Loaded talk 1472
Loaded talk 1473
Loaded talk 1474
Loaded talk 1475
Loaded talk 1476
Loaded talk 1477
Loaded talk 1478
Loaded talk 1479
Loaded talk 1480
Loaded talk 1481
Loaded talk 1482
Loaded talk 1483
Loaded talk 1484
Loaded talk 1485
Id 1486 not found.
Loaded talk 1487
Loaded talk 1488
Loaded talk 1489
Loaded talk 1490
Loaded talk 1491
Loaded talk 1492
Id 1493 not found.
Loaded talk 1494
Loaded talk 1495
Loaded talk 1496
Id 1497 not found.
Loaded talk 1498
Loaded talk 1499
Loaded talk 1500
Loaded talk 1501
Id 1502 not found.
Loaded talk 1503
Id 1504 not found.
Loaded talk 1505
Loaded talk 1506
Loaded talk 1507
Loaded talk 1508
Loaded talk 1509
Loaded talk 1510
Id 1511 not found.
Loaded talk 1512
Loaded talk 1513
Loaded talk 1514
Loaded talk 1515
Loaded talk 1516
Loaded talk 1517
Loaded talk 1518
Loaded talk 1519
Loaded talk 1520
Loaded talk 1521
Loaded talk 1522
Loaded talk 1523
Loaded talk 1524
Id 1525 not found.
Loaded talk 1526


Loaded talk 1947
Loaded talk 1948
Loaded talk 1949
Loaded talk 1950
Loaded talk 1951
Loaded talk 1952
Loaded talk 1953
Loaded talk 1954
Loaded talk 1955
Loaded talk 1956
Loaded talk 1957
Loaded talk 1958
Loaded talk 1959
Loaded talk 1960
Loaded talk 1961
Loaded talk 1962
Loaded talk 1963
Loaded talk 1964
Loaded talk 1965
Loaded talk 1966
Id 1967 not found.
Id 1968 not found.
Loaded talk 1969
Id 1970 not found.
Loaded talk 1971
Loaded talk 1972
Loaded talk 1973
Loaded talk 1974
Id 1975 not found.
Loaded talk 1976
Loaded talk 1977
Loaded talk 1978
Loaded talk 1979
Id 1980 not found.
Loaded talk 1981
Loaded talk 1982
Loaded talk 1983
Loaded talk 1984
Loaded talk 1985
Loaded talk 1986
Loaded talk 1987
Loaded talk 1988
Loaded talk 1989
Loaded talk 1990
Loaded talk 1991
Loaded talk 1992
Loaded talk 1993
Loaded talk 1994
Loaded talk 1995
Loaded talk 1996
Loaded talk 1997
Loaded talk 1998
Loaded talk 1999
Loaded talk 2000
Loaded talk 2001
Loaded talk 2002
Loaded talk 2003
Loaded talk 2004
Load

Loaded talk 2424
Loaded talk 2425
Loaded talk 2426
Loaded talk 2427
Loaded talk 2428
Loaded talk 2429
Loaded talk 2430
Loaded talk 2431
Id 2432 not found.
Loaded talk 2433
Loaded talk 2434
Loaded talk 2435
Loaded talk 2436
Loaded talk 2437
Loaded talk 2438
Loaded talk 2439
Loaded talk 2440
Loaded talk 2441
Loaded talk 2442
Loaded talk 2443
Id 2444 not found.
Loaded talk 2445
Id 2446 not found.
Loaded talk 2447
Loaded talk 2448
Loaded talk 2449
Id 2450 not found.
Loaded talk 2451
Loaded talk 2452
Loaded talk 2453
Loaded talk 2454
Loaded talk 2455
Loaded talk 2456
Loaded talk 2457
Loaded talk 2458
Loaded talk 2459
Loaded talk 2460
Loaded talk 2461
Loaded talk 2462
Loaded talk 2463
Loaded talk 2464
Loaded talk 2465
Loaded talk 2466
Loaded talk 2467
Loaded talk 2468
Loaded talk 2469
Loaded talk 2470
Loaded talk 2471
Loaded talk 2472
Loaded talk 2473
Loaded talk 2474
Loaded talk 2475
Loaded talk 2476
Loaded talk 2477
Loaded talk 2478
Loaded talk 2479
Loaded talk 2480
Loaded talk 2481
Loaded

Id 2900 not found.
Loaded talk 2901
Id 2902 not found.
Id 2903 not found.
Id 2904 not found.
Id 2905 not found.
Id 2906 not found.
Id 2907 not found.
Id 2908 not found.
Id 2909 not found.
Id 2910 not found.
Id 2911 not found.
Id 2912 not found.
Id 2913 not found.
Id 2914 not found.
Id 2915 not found.
Id 2916 not found.
Id 2917 not found.
Id 2918 not found.
Id 2919 not found.
Id 2920 not found.
Id 2921 not found.
Id 2922 not found.
Id 2923 not found.
Id 2924 not found.
Id 2925 not found.
Id 2926 not found.
Id 2927 not found.
Id 2928 not found.
Id 2929 not found.
Id 2930 not found.
Id 2931 not found.
Id 2932 not found.
Id 2933 not found.
Id 2934 not found.
Id 2935 not found.
Id 2936 not found.
Id 2937 not found.
Id 2938 not found.
Id 2939 not found.
Id 2940 not found.
Id 2941 not found.
Id 2942 not found.
Id 2943 not found.
Id 2944 not found.
Id 2945 not found.
Id 2946 not found.
Id 2947 not found.
Id 2948 not found.
Id 2949 not found.
Id 2950 not found.
Id 2951 not found.
Id 2952 not fo

Id 3332 not found.
Id 3333 not found.
Id 3334 not found.
Id 3335 not found.
Id 3336 not found.
Id 3337 not found.
Id 3338 not found.
Id 3339 not found.
Id 3340 not found.
Id 3341 not found.
Id 3342 not found.
Id 3343 not found.
Id 3344 not found.
Id 3345 not found.
Id 3346 not found.
Id 3347 not found.
Id 3348 not found.
Id 3349 not found.
Id 3350 not found.
Id 3351 not found.
Id 3352 not found.
Id 3353 not found.
Id 3354 not found.
Id 3355 not found.
Id 3356 not found.
Id 3357 not found.
Id 3358 not found.
Id 3359 not found.
Id 3360 not found.
Id 3361 not found.
Loaded talk 3362
Loaded talk 3363
Loaded talk 3364
Loaded talk 3365
Loaded talk 3366
Loaded talk 3367
Id 3368 not found.
Id 3369 not found.
Id 3370 not found.
Id 3371 not found.
Id 3372 not found.
Id 3373 not found.
Id 3374 not found.
Id 3375 not found.
Id 3376 not found.
Id 3377 not found.
Id 3378 not found.
Id 3379 not found.
Id 3380 not found.
Id 3381 not found.
Id 3382 not found.
Id 3383 not found.
Id 3384 not found.
Id 33

Id 3768 not found.
Id 3769 not found.
Id 3770 not found.
Id 3771 not found.
Id 3772 not found.
Id 3773 not found.
Id 3774 not found.
Id 3775 not found.
Id 3776 not found.
Id 3777 not found.
Id 3778 not found.
Id 3779 not found.
Id 3780 not found.
Id 3781 not found.
Id 3782 not found.
Id 3783 not found.
Id 3784 not found.
Id 3785 not found.
Id 3786 not found.
Id 3787 not found.
Id 3788 not found.
Id 3789 not found.
Id 3790 not found.
Id 3791 not found.
Id 3792 not found.
Id 3793 not found.
Id 3794 not found.
Id 3795 not found.
Id 3796 not found.
Id 3797 not found.
Id 3798 not found.
Id 3799 not found.
Id 3800 not found.
Id 3801 not found.
Id 3802 not found.
Id 3803 not found.
Id 3804 not found.
Id 3805 not found.
Id 3806 not found.
Id 3807 not found.
Id 3808 not found.
Id 3809 not found.
Id 3810 not found.
Id 3811 not found.
Id 3812 not found.
Id 3813 not found.
Id 3814 not found.
Id 3815 not found.
Id 3816 not found.
Id 3817 not found.
Id 3818 not found.
Id 3819 not found.
Id 3820 not 

KeyboardInterrupt: 

In [84]:
talk_sub.groupby('talk_id').count()

,id,transcript
talk_id,,
1.0,65,65
2.0,39,39
3.0,22,22
4.0,28,28
5.0,28,28
...,...,...
3642.0,12,12
3671.0,8,8
3680.0,35,35


In [93]:
talk_sub[(talk_sub.talk_id ==1) & (talk_sub.id == 0)].sort_values('id').values

array([[0.0, 1.0,
        "Thank you so much, Chris. to come to this stage twice;\nI'm extremely grateful. I have been blown away by this conference, and I want to thank all of you\nfor the many nice comments about what I had to say the other night. And I say that sincerely, partly because (Mock sob) I need that."]],
      dtype=object)

In [94]:
talk_meta

,name,tags,talk_id
0,Al Gore: Averting the climate crisis,"alternative energy,cars,climate change,culture...",1.0
1,Amy Smith: Simple designs to save a life,"MacArthur grant,alternative energy,design,engi...",2.0
2,Ashraf Ghani: How to rebuild a broken state,"business,corruption,culture,economics,entrepre...",3.0
3,Burt Rutan: The real future of space exploration,"NASA,aircraft,business,design,engineering,entr...",4.0
4,Chris Bangle: Great cars are great art,"art,business,cars,design,industrial design,inv...",5.0
...,...,...,...
2555,Niti Bhan: The hidden opportunities of the inf...,"Africa,business,creativity,culture,economics,i...",3642.0
2556,Matilda Ho: The future of good food in China,"TED Fellows,agriculture,china,entrepreneur,far...",3671.0
2557,Gretchen Carlson: How we can end sexual harass...,"activism,journalism,law,leadership,personal gr...",3680.0
2558,Dan Gartenberg: The brain benefits of deep sle...,"Human body,TED Residency,brain,health,memory,s...",3686.0


In [219]:
talk_data = talk_meta.set_index('talk_id').join(talk_sub.set_index('talk_id'), 'talk_id')

In [220]:
talk_data = talk_meta.set_index('talk_id').join(talk_sub.set_index('talk_id'), on = 'talk_id')
talk_meta = talk_meta[~talk_meta['talk_id'].isin(talk_data[talk_data['transcript'].isna()].index)]

In [95]:
with open(path.expanduser('~/work/data/talk_meta.pkl'), 'wb') as output:
    pickle.dump(talk_meta, output, pickle.HIGHEST_PROTOCOL)
with open(path.expanduser('~/work/data/talk_sub.pkl'), 'wb') as output:
    pickle.dump(talk_sub, output, pickle.HIGHEST_PROTOCOL)

In [2]:
from os import path
import pickle

with open(path.expanduser('~/work/data/talk_meta.pkl'), "rb") as input_file:
    talk_meta = pickle.load(input_file)
with open(path.expanduser('~/work/data/talk_sub.pkl'), "rb") as input_file:
    talk_sub = pickle.load(input_file)

In [6]:
talk_sub.groupby('talk_id').count().sort_values('transcript')

,id,transcript
talk_id,,
1263.0,1,1
1430.0,1,1
1677.0,1,1
2147.0,1,1
2273.0,2,2
...,...,...
2010.0,215,215
254.0,220,220
2209.0,230,230


In [4]:
talk_meta

,name,tags,talk_id
0,Al Gore: Averting the climate crisis,"alternative energy,cars,climate change,culture...",1.0
1,Amy Smith: Simple designs to save a life,"MacArthur grant,alternative energy,design,engi...",2.0
2,Ashraf Ghani: How to rebuild a broken state,"business,corruption,culture,economics,entrepre...",3.0
3,Burt Rutan: The real future of space exploration,"NASA,aircraft,business,design,engineering,entr...",4.0
4,Chris Bangle: Great cars are great art,"art,business,cars,design,industrial design,inv...",5.0
...,...,...,...
2573,Anjan Sundaram: Why I risked my life to expose...,"Africa,TED Fellows,corruption,journalism,war",3363.0
2574,Mike Kinney: A pro wrestler's guide to confidence,"business,creativity,humor,identity",3364.0
2575,Christen Reighter: I don't want children -- st...,"TEDx,children,family,identity,parenting,social...",3365.0
2576,"Chris Sheldrick: A precise, three-word address...","communication,cooperation,global development,i...",3366.0


In [8]:
!pip install nltk

     |████████████████████████████████| 1.5MB 1.4MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.4.5-cp37-none-any.whl size=1449906 sha256=b2e4b3acc85e60a303e4c883436e8657271fcb5cb33eccd90b5be9ad1163d56f
  Stored in directory: /home/jovyan/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk


In [16]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
def prepare_text(text):
    text = re.sub(r'\((.*?)\)', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text.lower())
    text = [w for w in text if not((w in ['thank', 'much']) or (w in nltk.corpus.stopwords.words('english')))]
    text = [nltk.corpus.wordnet.morphy(w) if nltk.corpus.wordnet.morphy(w) else w for w in text]
    return text

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
talk_sub[talk_sub.talk_id == 1]['transcript'].apply(lambda x: prepare_text(x)).values

array([list(['chris', 'truly', 'great', 'honor', 'opportunity', 'come', 'stage', 'twice', 'extremely', 'grateful', 'blow', 'away', 'conference', 'want', 'many', 'nice', 'comment', 'say', 'night', 'say', 'sincerely', 'partly', 'need']),
       list([]), list(['put', 'position']), list([]),
       list(['fly', 'air', 'force', 'two', 'eight', 'years']), list([]),
       list(['take', 'shoes', 'boot', 'get', 'airplane']), list([]),
       list([]),
       list(['tell', 'one', 'quick', 'story', 'illustrate', 'like']),
       list([]), list(['true', 'story', 'every', 'bit', 'true']),
       list(['soon', 'tipper', 'left', 'white', 'house']), list([]),
       list(['driving', 'home', 'nashville', 'little', 'farm', 'mile', 'east', 'nashville', 'driving']),
       list([]), list(['know', 'sound', 'like', 'little', 'thing']),
       list([]),
       list(['look', 'rear', 'view', 'mirror', 'sudden', 'hit', 'motorcade', 'back']),
       list([]), list(['hear', 'phantom', 'limb', 'pain']), list([])